In [1]:
import numpy as np

# Define the main function
def quasip():
    # Define global variables
    global h, alpha, beta, gamma, A, B, Omega, Picture, Map, k, b

    animsteps = 32

    # User input for choosing between map and ODE
    Map = input('Choose Map=1 for the map and Map=0 for the differential equation\nMap = ')
    if not Map.isdigit() or int(Map) not in [0, 1]:
        raise ValueError('You have to set Map to 0 or 1')
    Map = int(Map)

    print('Give now the desired parameters. Pressing enter will set the standard parameters')

    # Input parameters based on Map selection
    if Map == 0:
        alpha = input('alpha = ') or 1
        beta = input('beta = ') or 1.576
        gamma = input('gamma = ') or 1
        A = input('A = ') or 1.4
        B = input('B = ') or 1
        Omega = input('Omega = ') or 1.76
        Picture = input('Picture = ') or 1

        # Converting inputs to appropriate types
        alpha, beta, gamma, A, B, Omega, Picture = map(float, [alpha, beta, gamma, A, B, Omega]), int(Picture)
    else:
        Omega = input('Omega = ') or 0.292
        k = input('k = ') or 1
        b = input('b = ') or 0.25
        Picture = input('Picture = ') or 1

        # Converting inputs to appropriate types
        Omega, k, b, Picture = float(Omega), float(k), float(b), int(Picture)

    print('Press stop button to quit this run')

# Function call to start the program
quasip()


Give now the desired parameters. Pressing enter will set the standard parameters


ValueError: not enough values to unpack (expected 7, got 2)